In [8]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False :
  drive.mount('/content/gdrive')
  print('drive mounted')
else:
  print('already mounted')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
drive mounted


In [11]:
! ls /content/gdrive/My\ Drive

 공유폴더				    참가동의서.pdf
 교안및보고서				    retrain.py
'1018_SQL 전문가 가이드 2010 Edition.pdf'   sample_cardiac
 123.mp4				    segmentation
 1DCNN					    test.csv
 3DUnetCNN-master			    titanic
 advertise.tar.gz			    train.csv
'Colab Notebooks'			    Ultrasound
 foo.txt				    Untitled1.ipynb
 gender_submission.csv			    VLAD.ipynb
 generic_Unet				    workspace
 MyApp.gscript				    이동훈_입과증빙서류.zip


# [실습] 아래 그림의 Deep CNN 모델을 직접 만들어 봅시다.
### 블록 쌓기를 직접 하시면됩니다.
### 주의할 점은 크기와 차원 수!

<img width="726" alt="스크린샷 2019-05-09 오전 10 04 52" src="https://user-images.githubusercontent.com/11758940/57420313-e9a73180-7241-11e9-832e-a50145cbb8cd.png">

// ? 는 입력의 수, none과 같은 의미로 사용하였음
// FC : Fully Connected

conv(?, 28, 28, 32) => ReLU(?, 28, 28, 32) => MaxPool(?, 14, 14, 32) => Dropout => conv( ?, 14, 14, 32) => ReLU(?, 14, 14, 64) => MaxPool(?, 7, 7, 64) => Dropout => conv( ?, 7, 7, 128) => ReLU(?, 7, 7, 128) => MaxPool(?, 4, 4, 128) => Dropout() => Reshape(?, 2048) => FC(?,625) => FC(?,10)



- 최종으로 나온 값이 바로 hypothesis, logit 에 해당합니다. 
- 이를 이용하여 지금까지 해오시던 분류기를 작성하시면 됩니다.

In [20]:
import tensorflow as tf
import numpy as np


from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)
tf.set_random_seed(777)
tf.reset_default_graph()

X = tf.placeholder(tf.float32,[None,784])
X_img = tf.reshape(X,shape=[-1,28,28,1])
Y = tf.placeholder(tf.float32,[None,10])

learning_rate = 0.001
batch_size = 1000
total_epochs = 15
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L1 = tf.nn.dropout(L1,keep_prob=keep_prob)

W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L2 = tf.nn.dropout(L2,keep_prob=keep_prob)

W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev=0.01))
L3 = tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L3 = tf.nn.dropout(L3,keep_prob=keep_prob)
L3_flat = tf.reshape(L3,shape=[-1,4*4*128])

W4 = tf.get_variable('W4',shape=[4*4*128,625],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.matmul(L3_flat,W4)+b4

W5 = tf.get_variable('W5',shape=[625,10],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L4,W5)+b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(total_epochs):
  avg_cost = 0
  total_batch = int(mnist.train.num_examples/batch_size)
  for i in range(total_batch):
    batch_xs,batch_ys = mnist.train.next_batch(batch_size)
    c,_ = sess.run([cost,optimizer],feed_dict={X:batch_xs,Y:batch_ys,keep_prob:0.7})
    avg_cost += c/total_batch
  print('epoch : {} , cost : {} '.format(epoch+1,avg_cost))
  
prediction = tf.argmax(logits,1)
correct = tf.equal(prediction,tf.argmax(Y,1))

accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

print('accuracy : ',sess.run(accuracy, feed_dict={X:mnist.test.images,Y:mnist.test.labels,keep_prob:1}))



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
epoch : 1 , cost : 1.905485612695867 
epoch : 2 , cost : 0.3801077834584495 
epoch : 3 , cost : 0.17590140673247243 
epoch : 4 , cost : 0.11876340183344754 
epoch : 5 , cost : 0.09476334601640703 
epoch : 6 , cost : 0.08267149728807534 
epoch : 7 , cost : 0.06985826566815376 
epoch : 8 , cost : 0.06229216270148753 
epoch : 9 , cost : 0.05773062407970428 
epoch : 10 , cost : 0.054799489609219806 
epoch : 11 , cost : 0.05046835101463581 
epoch : 12 , cost : 0.046338407458229494 
epoch : 13 , cost : 0.042728771607984196 
epoch : 14 , cost : 0.04272005175325003 
epoch : 15 , cost : 0.04043497890233995 
accuracy :  0.9922


In [24]:
import random

r = random.randint(0,mnist.test.num_examples-1)
labels = tf.argmax(Y,1)
print('label :',sess.run(labels,feed_dict={Y:mnist.test.labels[r:r+1]}))
print('predict :',sess.run(prediction,feed_dict={X:mnist.test.images[r:r+1],keep_prob:1}))


label : [8]
predict : [8]


In [25]:
saver = tf.train.Saver()
saver.save(sess,'cnn_1')


'cnn_1'

In [26]:
! ls

checkpoint		   cnn_1.index	gdrive	    sample_data
cnn_1.data-00000-of-00001  cnn_1.meta	MNIST_data


In [0]:
saver = tf.train.Saver()
saver.save(sess,'/content/gdrive')